In [1]:
# Import pandas for working with tabular data
import pandas as pd

In [2]:
# Read exel file and get all the sheet_names
xl = pd.ExcelFile("DDH comparison with all plasmids.xlsx")
sheet_names = xl.sheet_names  # see all sheet names

In [3]:
# Filter only the sheet names that are bacteria strain datas
sheet_names = [sh for sh in sheet_names if sh.startswith("CP") or sh.startswith("AP")]

In [4]:
# Create a mapping between the strain_id_1 with all other strain ids and save the similarities

mapping = {}

for sheet_name in sheet_names:
    df = pd.read_excel(xl, sheet_name)

    df = df[['_1', 'Formula 2']]
    df = df[df['_1'].str.startswith('CP') | df['_1'].str.startswith('AP')]
    df.reset_index(drop=True, inplace=True)
    number_of_rows = len(df.index)

    sh_name = sheet_name.replace(" ", ".")
    mapping[sh_name] = {}

    for index, row in df.iterrows():
        strain_id = row['_1']
        similarity = row['Formula 2']

        mapping[sh_name][strain_id] = similarity

In [5]:
# Check that all the similarities are consistent(we don't have similarities that conflict with each other)
dupl_dict = {}

for strain1, mapping_dict in mapping.items():
  for strain2, similarity in mapping_dict.items():

    strain_names = [strain1, strain2]

    strain_names.sort()
    key = tuple(strain_names)

    if key in dupl_dict:
      dupl_dict[key].append(similarity)
    else:
      dupl_dict[key] = [similarity]

for k, v in dupl_dict.items():
    v = [float(val) for val in v]

    if len(set(v)) != 1:
        print("Eroare", set(v))
        print("Floats", v, set(v))

In [6]:
# Create a list of all unique strain ids
all_strains = set()

for strain1, mapping_dict in mapping.items():
  all_strains.add(strain1)
  for strain2, similarity in mapping_dict.items():
    all_strains.add(strain2)

all_strains = list(all_strains)

all_strains.sort()

In [7]:
# create an empty dataframe with strains on X and Y axes.
final_df = pd.DataFrame(columns=all_strains, index=all_strains)

In [8]:
# Fill in the similarities in the dataframe
for strain1, mapping_dict in mapping.items():
    index_name = strain1
    for strain2, similarity in mapping_dict.items():
        column_name = strain2
        value = float(similarity)

        final_df.at[index_name, column_name] = value

In [9]:
# Save the output dataframe
final_df.to_excel("dDDH.xlsx")